In [1]:
from fight import get_all_events_urls

all_events = get_all_events_urls()
print(len(all_events))

731


In [2]:
from fight import extract_fights_in_events

fights = extract_fights_in_events(all_events)
print(len(fights))

Extracting data from UFC 315: Muhammad vs. Della Maddalena on May 10, 2025
Time taken to extract event is 11.482510328292847
2


In [4]:
from fight import flatten_fight_data
import pandas as pd

fights_X, fights_y = fights

flattened_fight_dicts = []

for fight_dict in fights_X:
    flattened_fight_dicts.append(flatten_fight_data(fight_dict))
    
df = pd.DataFrame(flattened_fight_dicts)
df.to_csv("fights_df.csv")
print(df.head())

df = pd.DataFrame([fights_y]) # maybe add fight id
df.to_csv("fight_results.csv")
print(df.head())

In [5]:
import json
import numpy as np

df = pd.DataFrame(flattened_fight_dicts)
records = df.to_dict(orient="records")

with open("fights.json", "w") as f:
    json.dump(records, f, indent=2)

In [6]:
import pandas as pd

with open("fights_df.json", 'r+') as f:
    data = json.load(f)
    # current_df = pd.DataFrame(data)
old_df = pd.DataFrame(data)

print(old_df.shape)

(1219, 147)


**Fix/add the weights**

In [ ]:
import requests
from bs4 import BeautifulSoup
from fight import get_fight_details

with open("fights_df.json", 'r+') as f:
    data = json.load(f)
df = pd.DataFrame(data)



weights = []

for i in range(len(df)):
    fight_url = "http://www.ufcstats.com/fight-details/" + df.iloc[i]['Fight_id']
    response = requests.get(fight_url)
    fight_soup = BeautifulSoup(response.text, 'html.parser')
    fight_details = get_fight_details(fight_soup)
    weights.append(fight_details['division'])

df['Fighter_A_career_stats_weight'] = weights
df['Fighter_B_career_stats_weight'] = weights

In [8]:
df['Fighter_A_career_stats_weight'] = weights
df['Fighter_B_career_stats_weight'] = weights

In [9]:
fighter_id = "54f64b5e283b0ce7"

def get_fighter_mean_weight(df, fighter_id):
    
    a_weights = df.loc[df['Fighter_A_id'] == fighter_id, 'Fighter_A_career_stats_weight']
    b_weights = df.loc[df['Fighter_B_id'] == fighter_id, 'Fighter_B_career_stats_weight']
    all_weights = pd.concat([a_weights, b_weights])

    return np.mean(all_weights)

In [12]:
import numpy as np

print(np.mean(df.loc[df['Fighter_A_id'] == "54f64b5e283b0ce7", "Fighter_A_career_stats_weight"]))
print(df.shape)

def get_fights_weight_diff(df, real=True):
    if real:
        weight_mean_a = []
        weight_mean_b = []
    
        for i in range(len(df)):
            fighter_id = df.iloc[i]['Fighter_A_id']
            weight_mean_a.append(get_fighter_mean_weight(df, df.iloc[i]['Fighter_A_id']))
            weight_mean_b.append(get_fighter_mean_weight(df, df.iloc[i]['Fighter_B_id']))
        return np.array(weight_mean_a) - np.array(weight_mean_b)

    else:
        fighter_a_id = df['Fighter_A_id'].iloc[0]
        fighter_b_id = df['Fighter_B_id'].iloc[0]
        weight_mean_a = get_fighter_mean_weight(df, df.iloc[i]['Fighter_A_id'])
        weight_mean_b = get_fighter_mean_weight(df, df.iloc[i]['Fighter_B_id'])
        difference = weight_mean_a - weight_mean_b
        print(difference)
        return difference
        
    
df['Weight_diff'] = get_fights_weight_diff(df)
print(df['Weight_diff'])

with open('new_new_fights_df.json', "w") as file:
    json.dump(df.to_dict(orient="records"), file)

3.0
(1219, 149)
0       0.442857
1       0.071429
2       0.000000
3       0.000000
4       0.000000
          ...   
1214   -0.076923
1215    0.000000
1216    0.051282
1217    0.000000
1218   -0.750000
Name: Weight_diff, Length: 1219, dtype: float64
